### data-cleaning

This notebook was previously used to prototype our pre-processing functions.

It's currently used to load speeches.json and move things around.

It's a bit of a mess.

In [78]:
import pandas as pd
import json

In [79]:
# load the raw speeches db
#library = pd.read_json("library.json", orient='split')
#doc = pd.read_json("doc.json", orient="split").set_index("index")
speeches=pd.read_json('speeches.json')

In [80]:
speeches['item'] = speeches['items'].apply(lambda x: x[0]) 

In [81]:
# pull json out into separate columns
df = speeches.loc[:,'item'].to_frame()
df['candidate'] = df['item'].apply(lambda j: j['candidate'])
df['captions'] = df['item'].apply(lambda j: j['captions'])
df['videoId'] = df['item'].apply(lambda j: j['id'])
df['title'] = df['item'].apply(lambda j: j['snippet']['title'])
df['publishedAt'] = df['item'].apply(lambda j: j['snippet']['publishedAt'])
df['source'] = df['item'].apply(lambda j: j['snippet']['channelTitle'])

In [82]:
# drop duplicates
df = df.drop_duplicates(subset=['videoId'])

In [83]:
df.set_index('videoId')
df.rename(columns={'item': 'full_details'}, inplace=True)

In [84]:
# reorder cols
df = df[['videoId', 'full_details', 'candidate', 'captions', 'title',
       'publishedAt', 'source']]

In [85]:
df

,videoId,full_details,candidate,captions,title,publishedAt,source
0,zaaTZkqsaxY,"{'candidate': 'trump', 'captions': [{'duration...",trump,"[{'duration': 14.381, 'start': 4.504, 'text': ...",AXIOS on HBO: President Trump Exclusive Interv...,2020-08-04T03:57:59Z,HBO
1,gQi_sJxTtKA,"{'candidate': 'trump', 'captions': [{'duration...",trump,"[{'duration': 3.76, 'start': 0.0, 'text': '[Ap...",President Donald Trump's Full Republican Natio...,2020-08-28T22:42:26Z,NBC10 Philadelphia
3,RGualDMvwAo,"{'candidate': 'trump', 'captions': [{'duration...",trump,"[{'duration': 4.001, 'start': 0.399, 'text': '...",President Trump's Full RNC 2020 Speech | NowThis,2020-08-30T15:59:49Z,NowThis News
7,aEGm-ZXA6gM,"{'candidate': 'biden', 'captions': [{'duration...",biden,"[{'duration': 3.0, 'start': 14.92, 'text': 'so...",FULL VIDEO: Joe Biden delivers remarks on viol...,2020-08-31T18:11:49Z,Yahoo News
8,lLsae1X4gx4,"{'candidate': 'trump', 'captions': [{'duration...",trump,"[{'duration': 5.28, 'start': 6.560000000000000...",Jacob Blake protests: Trump announces funding ...,2020-09-01T19:54:33Z,Global News
9,9TLcgEq-zZo,"{'candidate': 'trump', 'captions': [{'duration...",trump,"[{'duration': 69.92, 'start': 14.92, 'text': '...","President Trump designates Wilmington, NC as t...",2020-09-02T18:54:58Z,The Hill
10,CiANHbSPyF8,"{'candidate': 'biden', 'captions': [{'duration...",biden,"[{'duration': 5.599, 'start': 1.76, 'text': 'g...",FULL VIDEO: Joe Biden makes remarks on educati...,2020-09-02T18:08:28Z,Yahoo News
11,SqNtc-t0Cr0,"{'candidate': 'trump', 'captions': [{'duration...",trump,"[{'duration': 3.33, 'start': 0.03, 'text': '[M...","FULL: Breaking down the Trump ""hot-mic"" tape w...",2020-09-03T16:59:53Z,Mother Jones
12,uBigQgyIPkE,"{'candidate': 'biden', 'captions': [{'duration...",biden,"[{'duration': 4.319, 'start': 57.92, 'text': '...",Joe Biden's full campaign speech HD 8/31/2020,2020-09-03T04:25:23Z,Mess of Media


In [77]:
#df.to_json("speeches2.json")

In [88]:
speeches = pd.read_json('speeches2.json')

In [97]:
# only add the new speeches to the existing "database" (speeches2)
speeches = speeches.append(df[~df.videoId.isin(speeches.index)])

In [98]:
# save database again
speeches.to_json("speeches2.json")

In [ ]:
## --- everything after this is unused ---

In [ ]:
# example
pd.DataFrame(YouTubeTranscriptApi.get_transcript('yy6jBRVDcpI'))

things to remove:
* (Applause)
* (Music)
* \[Applause\]
* \\n
* \>>

In [12]:
doc.sample(10)

,text,start,duration,videoId
index,,,,
919,one of the gravest threats to the,3407.900,5.160,Px2OG-ren-s
99,government to root out corruption in,332.460,6.960,sL1UF0KDwsE
865,public health briefing and an economic,2351.520,3.180,GXuEUeCH8hg
1032,can't get coded there's a mistrust of,2534.640,3.479,1lbMK2CG63k
166,[Applause],712.670,14.119,nLZCMmrkoZo
267,democracy they're written into our,967.329,6.031,FiPVOx-cAfQ
95,many many decades and on a nuclear way,459.660,7.230,VRwMzHNbzsE
627,should be willing to set up and,2232.390,3.780,xoBgzVWub2A
466,are not able to get accessible,1193.440,7.840,7S4hdImZXdw


In [21]:
# how long are these videos in minutes?
doc[["videoId",'duration']].groupby("videoId").sum()/60
# actually, some of the start times are bizarre. see yy6jBRVDcpI for example - starts at 97.596
# using sum() duration instead

,duration
videoId,
-8KiohxTJ0Y,23.620900
-VB24DMVUwY,59.704533
0Gl_Fl4KvHk,129.981450
0VpB4tllFkk,32.411433
0oX8uWNAgKs,80.711767
...,...
vz4HTSDlQ6g,61.742333
xoBgzVWub2A,91.795383
yGfh4eLgKYw,131.720200


In [22]:
doc.groupby("videoId").agg({'text': lambda x: ' '.join(x)})

,text
videoId,
-8KiohxTJ0Y,this my wife is my sister they switched on me ...
-VB24DMVUwY,">> The country is crying out\nfor leadership, ..."
0Gl_Fl4KvHk,American patriots and I can't wait to be back ...
0VpB4tllFkk,JOE BIDEN: Thanks an awful lot. I know that yo...
0oX8uWNAgKs,welcome everyone to our soul of the nation Sat...
...,...
vz4HTSDlQ6g,welcome everyone to 45 fast my name is Katrina...
xoBgzVWub2A,ladies and gentlemen please welcome to the sta...
yGfh4eLgKYw,down to Houston [Applause] every American it's...


index
0                 >>Good afternoon,\neveryone.
1           I apologize for\nthe slight delay.
2             We wanted to make\nsure everyone
3                        was able to get here.
4            Let me think the\nlocal officials
                        ...                   
280         we got 1.6 billion dollars we have
281    another 1.6 million we have a third 1.6
282     billion coming but we want to build it
283    quickly at one time we don't want to do
284                                   it peace
Name: text, Length: 88094, dtype: object

In [2]:
from datetime import datetime
# August 20, 1977 (the day NASA's Voyager 2 probe was launched) and November 5, 2018
datetime(2018, 11, 5) - datetime(1977, 8, 20)

datetime.timedelta(days=15052)

In [4]:
import pandas as pd
dates = pd.date_range('2020-04-01', '2020-06-01')

In [5]:
pd.DataFrame({'today': dates, 'yesterday': dates.shift(-1)})

,today,yesterday
0,2020-04-01,2020-03-31
1,2020-04-02,2020-04-01
2,2020-04-03,2020-04-02
3,2020-04-04,2020-04-03
4,2020-04-05,2020-04-04
...,...,...
57,2020-05-28,2020-05-27
58,2020-05-29,2020-05-28
59,2020-05-30,2020-05-29
60,2020-05-31,2020-05-30
